### K-means clustering

In [103]:
import os
from pathlib import Path
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans,DBSCAN
import nibabel as nib
from PIL import Image

In [104]:
NB_DIR = Path.cwd()
DATA_DIR = NB_DIR/'..'/'data'

In [105]:
img_list = sorted(glob(str(DATA_DIR/'2d_imgs'/'*')),key=os.path.getmtime)

In [106]:
#Source: Instruments & Data Tools
#Inspired from the Vector Quantization Example
#https://www.idtools.com.au/segmentation-k-means-python/
from sklearn import cluster
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label   
from skimage.morphology import remove_small_objects
def km_clust(array, n_clusters):
    
    # Create a line array, the lazy way
    X = array.reshape((-1, 1))
    # Define the k-means clustering problem
    k_m = cluster.KMeans(n_clusters=n_clusters, random_state=37, n_init=8) #,n_init=4)
    # Solve the k-means clustering problem
    k_m.fit(X)
    # Get the coordinates of the clusters centres as a 1D array
    values = k_m.cluster_centers_.squeeze()
    # Get the label of each point
    labels = k_m.labels_
    return(values, labels)

In [170]:
pred_list = []
for fn in img_list: 
    img = io.imread(fn,as_gray=True)
    img_flat = img.reshape((-1, 1))

    # Group similar grey levels using 8 clusters
    values, labels = km_clust(img, n_clusters = 2)
    values = [*range(len(values))]
    # Create the segmented array from labels and values
    img_segm = np.choose(labels, values)
    # Reshape the array as the original image
    img_segm.shape = img.shape
        
    pred_list.append(np.array(img_segm));

In [171]:
pred_mask = np.dstack(pred_list).astype(dtype=np.int16)
pred_mask.shape

(256, 256, 128)

In [188]:
img_data = nib.load(DATA_DIR/'kiwi_20050217191129_131073_RARE_high_res_axial.nii.gz')#.get_fdata()

In [189]:
pred_nii_mask = nib.Nifti1Image(pred_mask, affine=img_data.affine)

In [190]:
nib.save(pred_nii_mask, str(DATA_DIR/'pred_cluster_2_mask.nii.gz'))